In [88]:
# root_dir = "../outputs/exp2_init_models/exp2_init_models"
# root_dir = "../../outputs/exp1_all"
root_dir = "../../outputs/Exp3/exp3_merged"
# root_dir = "../../outputs/Exp3"
exp_name = "exp3"

In [89]:
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

def process_model_directories(root_dir, lista_de_modelos,exp_name):
    MAE_values = []
    RMSE_values = []
    SMAPE_values = []
    r2_values = []
    MASE_values = []
    training_time_values = []
    inference_time_values = []
    model_memory_values = []

    for subdir in os.listdir(root_dir):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path):
            for model_dir in lista_de_modelos:
                model_dir_path = os.path.join(subdir_path, model_dir)
                if os.path.isdir(model_dir_path):
                    if exp_name == "exp3":
                        try:
                            for file_name in os.listdir(model_dir_path):
                                if "model_metrics" in file_name and file_name.endswith('.csv'):
                                    model_metrics_path = os.path.join(model_dir_path, file_name)
                                    if os.path.isfile(model_metrics_path):
                                                model_metrics = pd.read_csv(model_metrics_path)
                                                rename_dict = {
                                                        'mae': 'MAE',
                                                        'rmse': 'RMSE',
                                                        'r2': 'r2',
                                                        'smape': 'SMAPE',
                                                        'mase': 'MASE',
                                                        'Training Time': 'Training_time',
                                                        'Inference Time': 'Inference_time',
                                                        'Model memory (MB)': 'Model memory (MB)'
                                                    }
                                                model_metrics.rename(columns=rename_dict, inplace=True)

                        except Exception as e:
                            print(f"An error occurred: {e} and {model_metrics_path} does not exist")
                    else:
                        model_metrics_path = os.path.join(model_dir_path, "model_data.csv")
                        model_metrics = pd.read_csv(model_metrics_path)
                    MAE_values.append(float(model_metrics.MAE))
                    RMSE_values.append(float(model_metrics['RMSE']))
                    SMAPE_values.append(float(model_metrics['SMAPE']))
                    r2_values.append(float(model_metrics['r2']))
                    MASE_values.append(float(model_metrics['MASE']))
                    training_time_values.append(float(model_metrics['Training_time']))
                    inference_time_values.append(float(model_metrics['Inference_time']))
                    model_memory_values.append(float(model_metrics['Model memory (MB)']))
        
    MAE_array = np.array(MAE_values)
    RMSE_array = np.array(RMSE_values)
    SMAPE_array = np.array(SMAPE_values)
    r2_array = np.array(r2_values)
    MASE_array = np.array(MASE_values)
    training_time_array = np.array(training_time_values)
    inference_time_array = np.array(inference_time_values)
    model_memory_array = np.array(model_memory_values)
    print(f"processed {len(model_memory_array)} seeds")
    return (MAE_array, RMSE_array, SMAPE_array, r2_array, MASE_array,
            training_time_array, inference_time_array, model_memory_array)
def arrays_to_dataframe(MAE_array, RMSE_array, SMAPE_array, r2_array, MASE_array,
                        training_time_array, inference_time_array, model_memory_array):
    SMALL_CONSTANT = 1
    log_training_time = np.log(training_time_array + SMALL_CONSTANT)
    log_inference_time = np.log(inference_time_array + SMALL_CONSTANT)

    # Create a DataFrame
    df = pd.DataFrame({
        'MAE': MAE_array,
        'RMSE': RMSE_array,
        'SMAPE': SMAPE_array,
        'r2': r2_array,
        'MASE': MASE_array,
        'Training_time': training_time_array,
        'Training_time_log': log_training_time,
        'Inference_time': inference_time_array,
        'Inference_time_log': log_inference_time,
        'Model_memory': model_memory_array
    })
    
    return df
import pandas as pd

def extract_window_size(model_name):
    parts = model_name.split('_ws_')
    if len(parts) > 1:
        try:
            window_size = int(parts[1])
            return parts[0], window_size
        except ValueError:
            pass
    return model_name, None

def add_window_size_column(df):
    if 'Window Size' not in df.columns:
        df[['Model', 'Window Size']] = df['Model'].apply(extract_window_size).apply(pd.Series)
    return df

def update_model_names(df, name_mapping):
    df['Model'] = df['Model'].replace(name_mapping)
    return df

## Generate csv for every seed

In [90]:
root_dir_exp1 = os.path.join(root_dir,"testbed_0")# because model names are inside seed folders
folder_names = []

# Iterate through each item in the directory
for item in os.listdir(root_dir_exp1):
    if os.path.isdir(os.path.join(root_dir_exp1, item)):

        folder_names.append(item)
folder_names

['Finetuned_128CL_True_RoPE_tested_on_64CL',
 'Finetuned_256CL_False_RoPE_tested_on_64CL',
 'Finetuned_32CL_False_RoPE_tested_on_32CL',
 'Finetuned_128CL_False_RoPE_tested_on_128CL',
 'Finetuned_128CL_False_RoPE_tested_on_256CL',
 'Finetuned_256CL_True_RoPE_tested_on_256CL',
 'Finetuned_32CL_True_RoPE_tested_on_32CL',
 'Finetuned_128CL_True_RoPE_tested_on_32CL',
 'Finetuned_32CL_False_RoPE_tested_on_256CL',
 'Finetuned_256CL_True_RoPE_tested_on_32CL',
 'Finetuned_64CL_True_RoPE_tested_on_64CL',
 'Finetuned_128CL_True_RoPE_tested_on_256CL',
 'Finetuned_32CL_False_RoPE_tested_on_128CL',
 'Finetuned_32CL_True_RoPE_tested_on_256CL',
 'Finetuned_128CL_False_RoPE_tested_on_64CL',
 'Finetuned_256CL_False_RoPE_tested_on_256CL',
 'Finetuned_64CL_True_RoPE_tested_on_32CL',
 'Finetuned_32CL_False_RoPE_tested_on_64CL',
 'Finetuned_64CL_True_RoPE_tested_on_256CL',
 'Finetuned_256CL_False_RoPE_tested_on_32CL',
 'Finetuned_128CL_True_RoPE_tested_on_128CL',
 'Finetuned_256CL_True_RoPE_tested_on_128CL'

In [108]:
import pandas as pd

def extract_window_size(model_name):
    parts = model_name.split('_ws_')
    if len(parts) > 1:
        try:
            return parts[0], int(parts[1])
        except ValueError:
            pass
    return model_name, None

def add_window_size_column(df):
    df[['Model', 'Window Size']] = df['Model'].apply(extract_window_size).apply(pd.Series)
    return df

def update_model_names(df, name_mapping):
    df['Model'] = df['Model'].replace(name_mapping)
    return df

# Initialize a list to collect dataframes
all_metrics = []

# Iterate through each model to process directories and collect metrics
for model_name in folder_names:
    lista_de_modelos = [f"{model_name}"]
    metrics_arrays = process_model_directories(root_dir, lista_de_modelos, exp_name)
    df = arrays_to_dataframe(*metrics_arrays)
    df["Model"] = model_name
    all_metrics.append(df)


all_models_df = pd.concat(all_metrics, ignore_index=True)

all_models_df.rename(columns={
    'Inference_time': 'Inference Time',
    'Training_time': 'Training Time',
    'Model_memory':'Model memory (MB)'
}, inplace=True)

all_models_df = add_window_size_column(all_models_df)
print(all_models_df.Model.unique())
if exp_name == "exp2":
    name_mapping = {
        'AdaptiveRandomForest': 'Adaptive Random Forest (ARF)',
        'HoeffdingAdaptiveTreeRegressor': 'Hoeffding Adaptive Tree Regressor',
        'HoeffdingTreeRegressor': 'Hoeffding Tree Regressor',
        'MLP_partialfit': 'MLP partialfit',
        'PassiveAggressive': 'Passive Aggressive (PA)',
        'SGDRegressor': 'SGD Regressor',
        'SRPRegressor': 'SRP Regressor',
        'XGBRegressor': 'XGBoost Regressor',
    }
    all_models_df = update_model_names(all_models_df, name_mapping)
elif exp_name=="exp1":
    name_mapping = {
        'SVR': 'Support Vector Regressor (SVR)',
        'XGBRegressor': 'XGBoost Regressor',
        'RandomForestRegressor': 'Random Forest',
        'LinearRegression': 'Linear Regression (LR)',
        'AdaBoostRegressor': 'Ada Boost Regressor',
        'DecisionTreeRegressor': 'Decision Tree Regressor',
        'MLPRegressor': 'MLP partialfit',
        'PassiveAggressiveRegressor': 'Passive Aggressive (PA)',
        'KNeighborsRegressor': 'K-Neighbors Regressor',
        'LSTM': 'LSTM',
        'GRU': 'GRU',
        'SGDRegressor': 'SGD Regressor',
        'BI-LSTM': 'BI-LSTM',
        'LSTM_ATTN': 'LSTM with Attention'
    }
    all_models_df = update_model_names(all_models_df, name_mapping)

    all_models_df = all_models_df[all_models_df['Model'] != 'MLP partialfit']
    all_models_df = all_models_df[all_models_df['Model'] != 'K-Neighbors Regressor']

all_models_df.to_csv(f"../../outputs/{exp_name}_full_results.csv", index=False)


processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 16 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 16 seeds
processed 20 seeds
processed 15 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 20 seeds
processed 15 seeds
processed 20 seeds
['Finetuned_128CL_True_RoPE_tested_on_64CL'
 'Finetuned_256CL_False_RoPE_tested_on_64CL'
 'Finetuned_32CL_False_RoPE_tested_on_32CL'
 'Finetuned_128CL_False_RoPE_tested_on_128CL'
 'Finetuned_128CL_False_RoPE_tested_on_256CL'
 'Finetuned_256CL_True_RoPE_tested_on_256CL'
 'Finetuned_32CL_True_RoPE_tested_on_32CL'
 'Finetuned_128CL_True_RoPE_tested_on_32CL'
 'Finetuned_32CL_False_RoPE_tested_on_256CL'
 'Finetuned_256CL_True_RoPE_tested_on_32CL'
 'Finetuned_64CL_True_

In [109]:
all_models_df

,MAE,RMSE,SMAPE,r2,MASE,Training Time,Training_time_log,Inference Time,Inference_time_log,Model memory (MB),Model,Window Size
0,3.979095,8.427499,23.472632,0.913205,1.125024,1470.660387,7.294147,76.969064,4.356312,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
1,5.061859,8.177308,24.805921,0.910219,1.605502,1467.349281,7.291894,76.777868,4.353857,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
2,4.735010,8.081811,24.093711,0.911998,1.305857,1478.667276,7.299573,76.599787,4.351565,0.316505,Finetuned_128CL_True_RoPE_tested_on_64CL,None
3,5.736139,10.110521,26.056445,0.867899,2.175479,1469.414234,7.293299,76.758397,4.353607,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
4,4.198985,8.101142,22.758916,0.917276,1.163660,1466.501148,7.291316,76.607342,4.351662,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
...,...,...,...,...,...,...,...,...,...,...,...,...
537,4.684654,9.123501,23.969622,0.895643,1.212260,1477.973678,7.299104,136.439170,4.923181,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
538,3.833669,7.984131,22.693442,0.923137,1.175680,1478.619176,7.299540,136.345406,4.922499,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
539,4.502005,8.037910,23.373216,0.917776,1.260558,1481.795525,7.301684,136.539286,4.923910,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
540,6.104835,11.565793,27.485174,0.829384,1.386435,1481.444956,7.301448,136.494026,4.923580,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None


In [110]:
all_models_df.Model.unique()

array(['Finetuned_128CL_True_RoPE_tested_on_64CL',
       'Finetuned_256CL_False_RoPE_tested_on_64CL',
       'Finetuned_32CL_False_RoPE_tested_on_32CL',
       'Finetuned_128CL_False_RoPE_tested_on_128CL',
       'Finetuned_128CL_False_RoPE_tested_on_256CL',
       'Finetuned_256CL_True_RoPE_tested_on_256CL',
       'Finetuned_32CL_True_RoPE_tested_on_32CL',
       'Finetuned_128CL_True_RoPE_tested_on_32CL',
       'Finetuned_32CL_False_RoPE_tested_on_256CL',
       'Finetuned_256CL_True_RoPE_tested_on_32CL',
       'Finetuned_64CL_True_RoPE_tested_on_64CL',
       'Finetuned_128CL_True_RoPE_tested_on_256CL',
       'Finetuned_32CL_False_RoPE_tested_on_128CL',
       'Finetuned_32CL_True_RoPE_tested_on_256CL',
       'Finetuned_128CL_False_RoPE_tested_on_64CL',
       'Finetuned_256CL_False_RoPE_tested_on_256CL',
       'Finetuned_64CL_True_RoPE_tested_on_32CL',
       'Finetuned_32CL_False_RoPE_tested_on_64CL',
       'Finetuned_64CL_True_RoPE_tested_on_256CL',
       'Finetuned_256C

In [111]:
# all_models_df[(all_models_df['Model'] == "Adaptive Random Forest (ARF)") & (all_models_df['Window Size'] == 64)][["MAE","Training Time","Inference Time"]]

In [112]:
all_models_df

,MAE,RMSE,SMAPE,r2,MASE,Training Time,Training_time_log,Inference Time,Inference_time_log,Model memory (MB),Model,Window Size
0,3.979095,8.427499,23.472632,0.913205,1.125024,1470.660387,7.294147,76.969064,4.356312,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
1,5.061859,8.177308,24.805921,0.910219,1.605502,1467.349281,7.291894,76.777868,4.353857,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
2,4.735010,8.081811,24.093711,0.911998,1.305857,1478.667276,7.299573,76.599787,4.351565,0.316505,Finetuned_128CL_True_RoPE_tested_on_64CL,None
3,5.736139,10.110521,26.056445,0.867899,2.175479,1469.414234,7.293299,76.758397,4.353607,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
4,4.198985,8.101142,22.758916,0.917276,1.163660,1466.501148,7.291316,76.607342,4.351662,0.316566,Finetuned_128CL_True_RoPE_tested_on_64CL,None
...,...,...,...,...,...,...,...,...,...,...,...,...
537,4.684654,9.123501,23.969622,0.895643,1.212260,1477.973678,7.299104,136.439170,4.923181,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
538,3.833669,7.984131,22.693442,0.923137,1.175680,1478.619176,7.299540,136.345406,4.922499,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
539,4.502005,8.037910,23.373216,0.917776,1.260558,1481.795525,7.301684,136.539286,4.923910,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None
540,6.104835,11.565793,27.485174,0.829384,1.386435,1481.444956,7.301448,136.494026,4.923580,0.316566,Finetuned_256CL_True_RoPE_tested_on_64CL,None


In [113]:
all_models_df.columns

Index(['MAE', 'RMSE', 'SMAPE', 'r2', 'MASE', 'Training Time',
       'Training_time_log', 'Inference Time', 'Inference_time_log',
       'Model memory (MB)', 'Model', 'Window Size'],
      dtype='object')

In [117]:
if exp_name != "exp3":

    all_models_df = all_models_df.dropna()

    summary = all_models_df.groupby(['Model', 'Window Size']).agg({
        'MAE': ['mean', 'std'],
        'RMSE': ['mean', 'std'],
        'SMAPE': ['mean', 'std'],
        'r2': ['mean', 'std'],
        'MASE': ['mean', 'std'],
        'Training Time': ['mean'],
        'Inference Time': ['mean'],
        'Model memory (MB)': ['mean'],
    }).reset_index()

    summary = summary.round(3)

    summary.columns = ['_'.join(col).strip() if col[1] else col[0] for col in summary.columns.values]


    summary = summary.rename(columns={
        'Model_': 'Model',
        'Window Size_': 'Window Size',
        'MAE_mean': 'MAE_mean',
        'MAE_std': 'MAE_std',
        'RMSE_mean': 'RMSE_mean',
        'RMSE_std': 'RMSE_std',
        'SMAPE_mean': 'SMAPE_mean',
        'SMAPE_std': 'SMAPE_std',
        'r2_mean': 'r2_mean',
        'r2_std': 'r2_std',
        'MASE_mean': 'MASE_mean',
        'MASE_std': 'MASE_std',
        'Training Time_mean': 'Training Time',
        'Inference Time_mean': 'Inference Time',
        'Model memory (MB)_mean': 'Model memory (MB)'
    })

    summary.to_csv(f"../../outputs/{exp_name}_model_metrics_avg.csv", index=False)

else:
    
    summary = all_models_df.groupby(['Model']).agg({
        'MAE': ['mean', 'std'],
        'RMSE': ['mean', 'std'],
        'SMAPE': ['mean', 'std'],
        'r2': ['mean', 'std'],
        'MASE': ['mean', 'std'],
        'Training Time': ['min', 'max'],#['mean'],
        'Inference Time': ['min', 'max'],#['mean'],
        'Model memory (MB)': ['mean'],
    }).reset_index()

    summary = summary.round(3)
    summary.to_csv(f"../../outputs/{exp_name}_model_metrics_avg.csv", index=False)

summary 

Model    MAE           RMSE         \
                                                 mean    std    mean    std   
0   Finetuned_128CL_False_RoPE_tested_on_128CL  3.668  0.189   7.229  0.402   
1   Finetuned_128CL_False_RoPE_tested_on_256CL  3.529  0.131   7.427  0.277   
2    Finetuned_128CL_False_RoPE_tested_on_32CL  5.439  0.670  11.500  0.708   
3    Finetuned_128CL_False_RoPE_tested_on_64CL  4.836  0.561  10.536  1.479   
4    Finetuned_128CL_True_RoPE_tested_on_128CL  3.608  0.216   7.102  0.473   
5    Finetuned_128CL_True_RoPE_tested_on_256CL  3.601  0.148   7.523  0.351   
6     Finetuned_128CL_True_RoPE_tested_on_32CL  5.600  0.805  11.877  1.061   
7     Finetuned_128CL_True_RoPE_tested_on_64CL  4.898  0.585   9.126  1.426   
8   Finetuned_256CL_False_RoPE_tested_on_128CL  3.764  0.152   7.756  0.541   
9   Finetuned_256CL_False_RoPE_tested_on_256CL  3.784  0.278   7.882  0.371   
10   Finetuned_256CL_False_RoPE_tested_on_32CL  7.183  0.882  13.224  1.286   
11   Finetuned_256CL_False_RoPE_tested_on_64CL  5.467  0.592  11.450  1.309   
12   Finetuned_256CL_True_RoPE_tested_on_128CL  3.777  0.182   7.567  0.389   
13   Finetuned_256CL_True_RoPE_tested_on_256CL  3.668  0.157   7.584  0.312   
14    Finetuned_256CL_True_RoPE_tested_on_32CL  5.204  0.564  11.938  1.391   
15    Finetuned_256CL_True_RoPE_tested_on_64CL  4.385  0.501   9.079  1.406   
16   Finetuned_32CL_False_RoPE_tested_on_128CL  4.018  0.216   7.138  0.285   
17   Finetuned_32CL_False_RoPE_tested_on_256CL  3.530  0.167   7.073  0.225   
18    Finetuned_32CL_False_RoPE_tested_on_32CL  5.166  0.553  10.522  0.415   
19    Finetuned_32CL_False_RoPE_tested_on_64CL  4.951  0.648   8.289  0.846   
20    Finetuned_32CL_True_RoPE_tested_on_128CL  4.161  0.329   7.277  0.429   
21    Finetuned_32CL_True_RoPE_tested_on_256CL  3.539  0.132   7.039  0.320   
22     Finetuned_32CL_True_RoPE_tested_on_32CL  5.386  0.626  10.697  0.558   
23     Finetuned_32CL_True_RoPE_tested_on_64CL  4.879  0.613   8.359  0.730   
24    Finetuned_64CL_True_RoPE_tested_on_128CL  4.027  0.450   7.276  0.433   
25    Finetuned_64CL_True_RoPE_tested_on_256CL  3.663  0.218   7.274  0.308   
26     Finetuned_64CL_True_RoPE_tested_on_32CL  5.210  0.436  10.603  0.600   
27     Finetuned_64CL_True_RoPE_tested_on_64CL  4.598  0.459   8.012  0.651   

     SMAPE            r2          MASE        Training Time            \
      mean    std   mean    std   mean    std           min       max   
0   22.710  0.555  0.938  0.007  1.054  0.108      1464.059  1479.320   
1   21.791  0.431  0.937  0.005  0.873  0.039      1462.988  1479.112   
2   25.210  1.234  0.826  0.022  1.691  0.172      1469.785  1481.935   
3   23.567  1.029  0.861  0.037  1.755  0.308      1466.388  1482.254   
4   22.370  0.675  0.940  0.008  1.068  0.059      1466.695  1593.816   
5   22.403  0.599  0.935  0.006  0.955  0.056      1461.805  1632.746   
6   24.876  1.489  0.821  0.034  1.909  0.310      1463.755  1537.669   
7   24.304  0.889  0.891  0.036  1.587  0.352      1466.501  1539.456   
8   22.754  0.626  0.929  0.009  0.988  0.057      1479.262  1540.526   
9   22.681  0.570  0.928  0.007  0.929  0.071      1478.983  1643.409   
10  28.479  1.709  0.759  0.048  1.940  0.271      1476.155  1548.370   
11  24.907  0.997  0.833  0.036  1.974  0.284      1477.296  1538.031   
12  22.713  0.537  0.933  0.007  1.046  0.072      1474.526  1487.614   
13  22.723  0.527  0.933  0.005  0.954  0.025      1477.663  1491.846   
14  24.619  1.047  0.826  0.042  1.892  0.301      1469.856  1487.298   
15  23.687  1.029  0.897  0.032  1.327  0.198      1475.882  1489.342   
16  23.173  0.536  0.939  0.006  1.084  0.097      1553.583  2207.728   
17  22.483  0.302  0.942  0.004  0.881  0.051      1556.432  2206.553   
18  24.334  0.931  0.850  0.014  2.100  0.213      1625.734  2218.965   
19  24.490  1.081  0.907  0.022  1.604  0.377      1556.777  2172.717   
20  23.508  0.532  0.936  0.008  1.096  0.100      1985.430  221

In [127]:
print(summary[["Training Time"]].mean())
print(summary[["Inference Time"]].mean())
print(summary[["Model memory (MB)"]].mean())

Training Time  min    1570.388857
               max    1850.087250
dtype: float64
Inference Time  min     89.308679
                max    107.533750
dtype: float64
Model memory (MB)  mean    0.316571
dtype: float64
